In [1]:
import pandas as pd
weather_set_dirty = pd.read_csv('inCSV/Weather_LosAngeles.csv', sep=",")

In [2]:
weather_set_dirty['timestamp'] = pd.to_datetime(weather_set_dirty['timestamp'])
mean_values_by_day = weather_set_dirty.groupby(weather_set_dirty["timestamp"].dt.date).mean(numeric_only=True).round(1)

In [3]:
for col in mean_values_by_day.columns:
    weather_set_dirty[col] = weather_set_dirty.apply(
        lambda row: mean_values_by_day.loc[row['timestamp'].date(), col]
        if pd.isnull(row[col]) else row[col],
        axis=1
    )

In [4]:
weather_set_dirty['cloud_cover_description'].fillna(method='ffill',inplace=True)
weather_set_dirty.to_csv('outCSV/Clean_Weather_Set.csv', index=False)

In [6]:
weather_set_dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77362 entries, 0 to 77361
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   city                     77362 non-null  object        
 1   timestamp                77362 non-null  datetime64[ns]
 2   temperature              77362 non-null  float64       
 3   cloud_cover              77362 non-null  float64       
 4   cloud_cover_description  77362 non-null  object        
 5   pressure                 77362 non-null  float64       
 6   windspeed                77362 non-null  float64       
 7   precipitation            77362 non-null  float64       
 8   felt_temperature         77362 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 5.3+ MB


In [2]:
bike_set_dirty = pd.read_csv('inCSV/metro_2017_2022.csv', sep=',', index_col=0)

In [8]:
bike_set_dirty.sort_values('start_station_id')
bike_set_dirty['start_station_lat'].fillna(method="ffill", inplace=True)
bike_set_dirty['start_station_lon'].fillna(method="ffill", inplace=True)

bike_set_dirty.sort_values('end_station_id')
bike_set_dirty['end_station_lat'].fillna(method="ffill", inplace=True)
bike_set_dirty['end_station_lon'].fillna(method="ffill", inplace=True)

In [9]:
bike_set_dirty_agg_start = bike_set_dirty.groupby('start_station_id')[['start_station_lat', 'start_station_lon']].agg(lambda x: x.mode().iloc[0])
bike_set_dirty_agg_end = bike_set_dirty.groupby('end_station_id')[['end_station_lat', 'end_station_lon']].agg(lambda x: x.mode().iloc[0])

In [10]:
bike_set_dirty_merge = bike_set_dirty.merge(bike_set_dirty_agg_start, on='start_station_id', suffixes=('','_mode'))
bike_set_dirty_merged = bike_set_dirty_merge.merge(bike_set_dirty_agg_end, on='end_station_id', suffixes=('','_mode'))
bike_set_dirty_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   start_time              1552782 non-null  object 
 1   end_time                1552782 non-null  object 
 2   start_station_id        1552782 non-null  int64  
 3   start_station_lat       1552782 non-null  float64
 4   start_station_lon       1552782 non-null  float64
 5   end_station_id          1552782 non-null  int64  
 6   end_station_lat         1552782 non-null  float64
 7   end_station_lon         1552782 non-null  float64
 8   bike_id                 1552782 non-null  object 
 9   start_station_lat_mode  1552782 non-null  float64
 10  start_station_lon_mode  1552782 non-null  float64
 11  end_station_lat_mode    1552782 non-null  float64
 12  end_station_lon_mode    1552782 non-null  float64
dtypes: float64(8), int64(2), object(3)
memory usage: 165.9+ M

In [11]:
from datetime import datetime
dateformat0 = "%H/%M/%S %d-%m-%Y"
dateformat1 = "%Y-%m-%d %H:%M:%S"
for i, row in bike_set_dirty_merged.iterrows():
    bike_set_dirty_merged.at[i, 'start_station_lat'] = bike_set_dirty_merged.loc[i, 'start_station_lat_mode']
    bike_set_dirty_merged.at[i, 'start_station_lon'] = bike_set_dirty_merged.loc[i, 'start_station_lon_mode']
    bike_set_dirty_merged.at[i, 'end_station_lat'] = bike_set_dirty_merged.loc[i, 'end_station_lat_mode']
    bike_set_dirty_merged.at[i, 'end_station_lon'] = bike_set_dirty_merged.loc[i, 'end_station_lon_mode']

    try:
        bike_set_dirty_merged.at[i, 'start_time'] = datetime.strptime(bike_set_dirty_merged.loc[i, 'start_time'], dateformat0)
    except ValueError:
        bike_set_dirty_merged.at[i, 'start_time'] = datetime.strptime(bike_set_dirty_merged.loc[i, 'start_time'], dateformat1)
    try:
        bike_set_dirty_merged.at[i, 'end_time'] = datetime.strptime(bike_set_dirty_merged.loc[i, 'end_time'], dateformat0)
    except ValueError:
        bike_set_dirty_merged.at[i, 'end_time'] = datetime.strptime(bike_set_dirty_merged.loc[i, 'end_time'], dateformat1)

In [12]:
bike_set_almost_clean = bike_set_dirty_merged
bike_set_almost_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   start_time              1552782 non-null  object 
 1   end_time                1552782 non-null  object 
 2   start_station_id        1552782 non-null  int64  
 3   start_station_lat       1552782 non-null  float64
 4   start_station_lon       1552782 non-null  float64
 5   end_station_id          1552782 non-null  int64  
 6   end_station_lat         1552782 non-null  float64
 7   end_station_lon         1552782 non-null  float64
 8   bike_id                 1552782 non-null  object 
 9   start_station_lat_mode  1552782 non-null  float64
 10  start_station_lon_mode  1552782 non-null  float64
 11  end_station_lat_mode    1552782 non-null  float64
 12  end_station_lon_mode    1552782 non-null  float64
dtypes: float64(8), int64(2), object(3)
memory usage: 198.1+ M

In [ ]:
#from datetime import datetime
#dateformat0 = "%H/%M/%S %d-%m-%Y"
#dateformat1 = "%Y-%m-%d %H:%M:%S"
#for i, row in bike_set_almost_clean.iterrows():
#    try:
#        bike_set_almost_clean.at[i, 'start_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'start_time'], dateformat0)
#    except ValueError:
#        bike_set_almost_clean.at[i, 'start_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'start_time'], dateformat1)
#    try:
#        bike_set_almost_clean.at[i, 'end_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'end_time'], dateformat0)
#    except ValueError:
#        bike_set_almost_clean.at[i, 'end_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'end_time'], dateformat1)
#bike_set_almost_clean.head()

In [13]:
bike_set_clean = bike_set_almost_clean.drop(['start_station_lat_mode', 'start_station_lon_mode', 'end_station_lat_mode', 'end_station_lon_mode'], axis=1)
bike_set_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   start_time         1552782 non-null  object 
 1   end_time           1552782 non-null  object 
 2   start_station_id   1552782 non-null  int64  
 3   start_station_lat  1552782 non-null  float64
 4   start_station_lon  1552782 non-null  float64
 5   end_station_id     1552782 non-null  int64  
 6   end_station_lat    1552782 non-null  float64
 7   end_station_lon    1552782 non-null  float64
 8   bike_id            1552782 non-null  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 150.7+ MB


In [14]:
bike_set_clean['start_time'] = pd.to_datetime(bike_set_almost_clean['start_time'], format="%Y-%m-%d %H:%M:%S")
bike_set_clean.info()
bike_set_clean['end_time'] = pd.to_datetime(bike_set_almost_clean['end_time'], format="%Y-%m-%d %H:%M:%S")
bike_set_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   start_time         1552782 non-null  datetime64[ns]
 1   end_time           1552782 non-null  object        
 2   start_station_id   1552782 non-null  int64         
 3   start_station_lat  1552782 non-null  float64       
 4   start_station_lon  1552782 non-null  float64       
 5   end_station_id     1552782 non-null  int64         
 6   end_station_lat    1552782 non-null  float64       
 7   end_station_lon    1552782 non-null  float64       
 8   bike_id            1552782 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 150.7+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype         
---  ----

In [15]:
bike_set_clean.to_csv('outCSV/Clean_Metro_Set.csv')